네이버에서 ETF 종목 추출

In [ ]:
# import sys
import csv
import requests
import json
import pandas as pd
import datetime

today = datetime.date.today().strftime("%y%m%d")
filename = today + "_etf_code.csv"
f = open(filename, "w", encoding="euc-kr", newline="")

url = 'https://finance.naver.com/api/sise/etfItemList.nhn'
json_data = json.loads(requests.get(url).text)
df = pd.json_normalize(json_data['result']['etfItemList'])

df.to_csv(f)
f.close()
# df.to_excel('etf_list_20251030.xlsx')

print("save")


save


기간별 수익률

In [ ]:
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import json

def fn_etf_data(code, name):
    url = "http://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A"+code+"&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=701"
    #print(url)
    
    res = requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, 'lxml')    
    #print(res.text)
    #sys.exit()
    script = soup.find_all("script")[7].text
    #print(script)
    
    url_su = "http://comp.fnguide.com/SVO2/xml/Snapshot_all/{}.xml".format(code)
    
    res_su = requests.get(url_su)
    res_su.raise_for_status()
    soup_su = BeautifulSoup(res_su.text, 'xml')  
    #print(soup_su)
    
    try:        
        #s_name = soup.find("price").find("close_val").text.replace(u'\xa0','')
        #s_name = " "
        #s_name = soup.shareholders_type.get_text()
        #/html/body/div[1]/xml/upjong_B/record[1]/item_nm/text()

        s_code = "e=" + code
        s_name = name

        s_price = soup_su.find("close_val").text
        
        s_max = soup_su.find("high52week").text
        s_min = soup_su.find("low52week").text
        
        #기간별 수익률
        s_1m = soup_su.find("change_1month").text
        s_3m = soup_su.find("change_3month").text
        s_6m = soup_su.find("change_6month").text
        s_1y = soup_su.find("change_12month").text        
        s_3y = soup_su.find("change_36month").text

        s_mksum = soup_su.find("market_sum").text
                
        ####################################################

        # etf1FundInfoData 배열 부분만 추출
        pattern = re.compile(r"var\s+etf1FundInfoData\s*=\s*(\[.*?\]|\{.*?\});", re.DOTALL)
        match = pattern.search(script)
        #print(script)
        #print(match)

        if match:
            array_text = match.group(1)

            # 3. HTML 태그(<span ...>) 제거
            clean_text = re.sub(r"<.*?>", "", array_text)

            # 4. JSON으로 변환
            data = json.loads(clean_text)

            # "val01"을 키로, "val02"를 값으로 딕셔너리화
            # etf_dict = {}
            # for item in data2:
            #     etf_dict[item["val01"]] = item["val02"]
            #     etf_dict[item["val03"]] = item["val04"]

            # 배열일 수도 있고 객체일 수도 있음
            if isinstance(data, list):
                etf_dict = {item["val01"]: item["val02"] for item in data}
                for item in data:
                    etf_dict[item["val03"]] = item["val04"]
            elif isinstance(data, dict):
                etf_dict = {
                    data["val01"]: data["val02"],
                    data["val03"]: data["val04"]
                }

            #print(etf_dict)

            s_fee = etf_dict.get("총보수")
            s_setupday = etf_dict.get("설정일")
            s_setupamt = etf_dict.get("설정액/순자산").split("/")
              
        #분배금
        s_b1 = soup.find("div", attrs={"id":"etfDivInfo1"}).find_all("td")[0].text
        s_b2 = soup.find("div", attrs={"id":"etfDivInfo1"}).find_all("td")[1].text
        s_b3 = soup.find("div", attrs={"id":"etfDivInfo1"}).find_all("td")[2].text
        s_b4 = soup.find("div", attrs={"id":"etfDivInfo1"}).find_all("td")[3].text
        #//*[@id="etfDivInfo1"]/table/tbody/tr[1]/td[1]
        # print(s_b1)
        
        #펀드현황 및 분배금
        
        r_data = [s_name, s_code, s_price, s_1m, s_3m, s_6m, s_1y, s_3y, s_max, s_min, s_mksum, s_fee, s_setupday, s_setupamt[0], s_setupamt[1], s_b1, s_b2, s_b3, s_b4]
        #r_data = [s_name, s_code, s_price, s_1m, s_3m, s_6m, s_1y, s_3y, s_max, s_min]

        #print(r_data)
        return r_data
    
    except:
        #s_name = ""
        s_b1 = ""
        s_b2 = ""
        s_b3 = ""
        s_b4 = ""
        
        s_1m = ""
        s_3m = ""
        s_6m = ""
        s_1y = ""
        s_3y = ""
        s_max = ""
        s_min = ""
        
        print(s_name+" error")        


    #print("---------------------------------------")

In [ ]:
import datetime

#today = datetime.date.today().strftime("%y%m%d")
today = "251208"

filename = today + "_etf_period.csv"
f = open(filename, "w", encoding="euc-kr", newline="")
writer = csv.writer(f)

title = "종목명,종목코드,현재가,1개월,3개월,6개월,1년,3년,최고가,최저가,시가총액,총보수,설정일,설정액,순자산,최근분배금,지급일자,연분배횟수,배당수익률".split(",")

#print(type(title))
writer.writerow(title)


# 종목
#codes = ['005930', '005380', '012330', '105560', '033780', '000810', '001040', '294870', '237690', '086450', '230360', '183490', '030520', '089030', '095340', '000640', '073490', '010130', '035420', '035720']

#df1 = pd.read_csv('D:/정리 2020/작업/fn가이드/수집종목.csv', encoding="euc-kr")

file_etf = today + "_etf_code.csv"
df_etf = pd.read_csv(file_etf, encoding="euc-kr")
df_etf_select = df_etf.loc[(df_etf['marketSum'] > 300) & (df_etf['quant'] > 10000)]
codes = df_etf_select['itemcode']
names = df_etf_select['itemname']

#print(codes)
i = int()

for code, name in zip(codes, names):
    #print(code[2:])
    i = i + 1
    if i%100==0 : print(i)
    data = fn_etf_data(code, name)   
    if data :
        #print(code)
        #print(data)        
        writer.writerow(data)

print('save')
f.close()

관심종목 실시간 조회

In [ ]:
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup

def naver_data(code):
    url = "https://finance.naver.com/item/main.nhn?code=" + code    
    res =requests.get(url)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")

    try :
        s_name = soup.find('div',{'class':'h_company'}).find('h2').text.replace(u'\xa0','')        
        s_price = soup.find('p',{'class':'no_today'}).find('span',{'class':'blind'}).text.replace(u'\xa0','')        
        s_ch = soup.find('p',{'class':'no_exday'}).find_all('em')[0].find('span',{'class':'blind'}).text.replace(u'\xa0','')        
        s_ch1 = soup.find('p',{'class':'no_exday'}).find_all('em')[1].find('span',{'class':'ico'}).text.replace(u'\xa0','')        
        s_ch2 = soup.find('p',{'class':'no_exday'}).find_all('em')[1].find('span',{'class':'blind'}).text.replace(u'\xa0','')        
        #s_pr1 = soup.find('dl',{'class':'blind'}).find_all('dd')[4].text.replace(u'\xa0','')
        #s_pr2 = soup.find('dl',{'class':'blind'}).find_all('dd')[5].text.replace(u'\xa0','')
        s_pr3 = soup.find('dl',{'class':'blind'}).find_all('dd')[6].text.replace(u'\xa0','')[3:]        
        s_pr4 = soup.find('dl',{'class':'blind'}).find_all('dd')[8].text.replace(u'\xa0','')[3:]        
        s_siga = soup.find('div',{'class':'first'}).find('table').find_all('tr')[0].find('td').find('em').text.strip().replace(u'\t','').replace(u'\n','')        
        #s_fore = soup.find('div',{'class':'gray'}).find('table').find_all('tr')[2].find('td').find('em').text.strip().replace(u'\t','').replace(u'\n','')        
        s_52h = soup.find('table',{'summary':'시가총액 정보'}).find_all('tr')[2].find('td').find_all('em')[0].text        
        s_52l = soup.find('table',{'summary':'시가총액 정보'}).find_all('tr')[2].find('td').find_all('em')[1].text        
        #s_target = soup.find('table',{'class':'rwidth'}).find_all('tr')[0].find('td').find_all('em')[1].text
        #s_bae = soup.find('table',{'class':'per_table'}).find_all('tr')[3].find('td').find('em').text
        
        r_data = [s_name, s_price, s_ch, s_ch1, s_ch2, s_pr3, s_pr4, s_siga, s_52h, s_52l]

        #print(r_data)
        return r_data
    
    except:
        print('no data'+code)

In [ ]:
df_inter = pd.read_csv('interest_etf.csv', encoding="euc-kr")

codes = df_inter['종목코드']

#i = int()
list_realtime = []

for code in codes:
    #print(code[2:])
    #i = i + 1
    #print(i)
    data1 = naver_data(code[2:])
    
    try:
        list_realtime.append(
            {
            '종목명'  :data1[0].replace('\n',''),
            '현재가'  :data1[1],
            '변동금액':int(data1[2].replace(',','')),
            '손익'    :data1[3],
            '비율'    :float(data1[4]),
            '고가'    :data1[5],
            '저가'    :data1[6],
            '시총'    :data1[7],            
            '52H'    :data1[8],
            '52L'    :data1[9]
            })
        
    except:
        print("리스트 입력오류")

# display(list_realtime)
df_realtime = pd.DataFrame(list_realtime)
df_realtime_sort = df_realtime.sort_values(by=['손익','비율','변동금액'], ascending=False)

display(df_realtime_sort)   

https://finance.naver.com/item/main.nhn?code=475720
입력완료
https://finance.naver.com/item/main.nhn?code=490590
입력완료
https://finance.naver.com/item/main.nhn?code=491620
입력완료
https://finance.naver.com/item/main.nhn?code=472150
입력완료
https://finance.naver.com/item/main.nhn?code=148020
입력완료
https://finance.naver.com/item/main.nhn?code=379800
입력완료
https://finance.naver.com/item/main.nhn?code=0066W0
입력완료
https://finance.naver.com/item/main.nhn?code=272580
입력완료
https://finance.naver.com/item/main.nhn?code=379810
입력완료
https://finance.naver.com/item/main.nhn?code=283580
입력완료
https://finance.naver.com/item/main.nhn?code=091160
입력완료
https://finance.naver.com/item/main.nhn?code=232080
입력완료
https://finance.naver.com/item/main.nhn?code=091180
입력완료
https://finance.naver.com/item/main.nhn?code=157500
입력완료
https://finance.naver.com/item/main.nhn?code=091170
입력완료
https://finance.naver.com/item/main.nhn?code=140700
입력완료
https://finance.naver.com/item/main.nhn?code=453870
입력완료
https://finance.naver.com/item/

,종목명,현재가,변동금액,손익,비율,고가,저가,시총,52H,52L
19,TIGER 조선TOP10,"27,680",455,-,1.62,"28,165","27,600","7,252","32,595","10,520"
22,TIGER 지주회사,"16,350",260,-,1.57,"16,635","16,330","3,172","17,215","9,035"
15,KODEX 보험,"12,335",195,-,1.56,"12,530","12,335",530,"13,500","8,280"
21,KODEX 에너지화학,"13,620",210,-,1.52,"13,930","13,600",305,"14,845","8,390"
14,KODEX 은행,"12,555",135,-,1.06,"12,745","12,520","3,597","13,340","7,565"
13,TIGER 증권,"10,110",105,-,1.03,"10,250","10,090","1,965","11,445","5,540"
25,PLUS 자사주매입고배당주,"10,970",80,-,0.72,"11,080","10,960",998,"11,130","9,670"
24,RISE AI전력인프라,"12,755",55,-,0.43,"12,910","12,735",523,"14,830","9,485"
12,KODEX 자동차,"25,045",55,-,0.22,"25,210","24,730","4,383","25,890","16,500"
1,RISE 미국AI밸류체인데일리고정커버드콜,"13,720",30,-,0.22,"13,765","13,680","4,480","14,935","7,715"


In [39]:
rr_data = ['a','b']
print(rr_data)

['a', 'b']
